In [15]:
import numpy as np
import re
import sys
import itertools

In [16]:

class AlgoSearch():
    expansions_max_lengths ={}
    #expansions_indexes = {}
    expansions_letters = []
    expansions_indexes = []
    solutions=[]
    expansions_prune = {}
    
    words_to_expand = []
    
    dict_exp = {}
    old_dict_exp = {}
    letter_to_index = {}
    s= ""
    found = False
    

        
    def test_words(self, solution):
        
        for w in self.words_to_expand:
            
            expanded_word = ""
            
            for char in w:
                expanded_word += self.get_char(char,solution)
            
            #print(expanded_word)
            if(expanded_word not in self.s):
                return False
            
        return True
    
    def impossible_index(self,solution):
        
        for w in self.words_to_expand:
            
            expanded_word = ""
            
            for char in w:
                expanded_word += self.get_char(char,solution)
            
            #print(expanded_word)
            if(expanded_word not in self.s):
                return False
            
        return True
            
    def get_char(self,char, solution):
        if(char in self.letter_to_index):
            idx= self.letter_to_index[char]
            selected = solution[idx]
            return self.dict_exp[char][selected]
        else:
            return char

    def increment_index(self,letter):
        expansions_indexes[letter] = expansions_indexes[letter]+1
        
    def recursion(self, expanded):
        og_exp = expanded.copy()
        for i in range(len(expanded)):
            expanded_copy = og_exp.copy()
            is_solution = self.test_words(expanded_copy)
            
            if is_solution:
                self.print_solution(expanded_copy)
                self.found = True
                return
            expanded_copy[i] = expanded_copy[i]+1
            

            
            if (expanded_copy[i]> len(self.dict_exp[self.expansions_letters[i]])-1):
                return

            if(str(expanded_copy) not in self.expansions_prune):
                self.solutions.append(expanded_copy)
                self.expansions_prune[str(expanded_copy)] = 1
                self.recursion(expanded_copy)
                
    def print_solution(self,solution):
        for idx in range(len(solution)):
            
            letter = self.expansions_letters[idx]
            if(len(self.dict_exp[letter]) > 0):
                print("{}:{}".format(letter,self.dict_exp[letter][solution[idx]]))
            else:
                print("{}:{}".format(letter,self.old_dict_exp[letter][solution[idx]]))
                
    def preprocess_expansions(self):
        self.old_dict_exp = self.dict_exp.copy()
        for letter in self.expansions_letters:

            to_pop = []
            
            is_in_expansions=False
            for w in self.words_to_expand:
                if letter in w:
                    is_in_expansions=True
            
            if is_in_expansions:
                for idx in range(len(self.dict_exp[letter])):

                        # its A for the first index i
                        #print(exp_copy)
                        #print(letter_to_expand)
                        to_pop_bool = False

                        for w in self.words_to_expand:
                            #print(w)

                            new_word = ""

                            for c in w:
                                if(c==letter):
                                    new_word += self.dict_exp[letter][idx]
                                else:
                                    new_word += "."

                            if re.search(new_word, self.s):
                                #print("match_possible")
                                None
                            else:
                                to_pop_bool = True


                        if to_pop_bool == True:
                            to_pop.append(idx)
                self.dict_exp[letter] = [i for j, i in enumerate(self.dict_exp[letter]) if j not in to_pop]
            else:
                self.dict_exp[letter] = []
                
    def load_file(self,info):
        ##Used for codejudge
        infos = info.split("\n")
        k = int(infos[0])
        s = infos[1]

        self.s = s

        for i in range(2,2+k):
            self.words_to_expand.append(infos[i])

        index_letter = 0
        for i in range(2+k,len(infos)):
            exp = infos[i]
            if exp:

                letter = exp[0]
                possibilities = exp[2:]
                self.dict_exp[letter] = possibilities.split(",")
                self.dict_exp[letter] = possibilities.split(",")
                self.expansions_max_lengths[letter]  = len(possibilities.split(","))-1
                self.letter_to_index[letter]=index_letter
                #pointers.expansions_indexes[letter] = 0
                self.expansions_letters.append(letter)
                self.expansions_indexes.append(0)
                index_letter += 1

In [17]:
def decode(content):
    lines = content.split('\n')
    k = int(lines[0])
    s = lines[1]

    if re.match("[a-z]+", s) is None:
        return None

    ts = []
    t_pattern = re.compile('^[a-zA-Z]+$')
    for t_string in lines[2:2+k]:
        match = re.match(t_pattern, t_string)
        if match is None:
            continue
        ts.append(t_string)

    if len(ts) != k:
        return None
    
    Rs = dict()
    R_pattern = re.compile('^([A-Z]+):([a-z]+(,[a-z]+)*)$')
    for R_string in lines[2+k:]:
        match = re.match(R_pattern, R_string)
        if match is None:
            continue
        letter = match.group(1)
        possibilities = match.group(2).split(',')
        Rs[letter] = possibilities
    
    return (k, s, ts, Rs)

def reduce_instance(instance):
    k, s, ts, Rs = instance

    ts_joined = ''.join(ts)
    gammas_used = set(re.findall('([A-Z])', ts_joined))

    gammas_removed = []

    Rs_new = dict()
    for key in Rs.keys():

        # Skip symbols that are not used
        if key not in gammas_used:
            gammas_removed.append(key)
            continue

        rs_new = [r for r in Rs[key] if r in s]
        Rs_new[key] = rs_new

    return (k, s, ts, Rs_new), gammas_removed

def replace_gammas(t, assignment):
    # Replace capital letters with new assignment
    gammas = re.findall('([A-Z])', t)
    t_new = t
    for gamma in gammas:
        t_new = t_new.replace(gamma, assignment[gamma])
    return t_new

def assignment_is_solution(ts, R_new, s):
    
    for t in ts:
            
        t_new = replace_gammas(t, R_new)

        # Check that t_new is substring of s
        if t_new not in s:
            return False

    return True

def find_brute_force_solution(instance):
    k, s, ts, Rs = instance

    keys = Rs.keys()
    values = [Rs[key] for key in keys]

    if any(len(vs) == 0 for vs in values):
        return None

    for R_combination in itertools.product(*values):

        # Construct new R-assignment dictionary
        assignment = {key:R_combination[idx] for idx, key in enumerate(keys)}

        if assignment_is_solution(ts, assignment, s):
            return assignment
        
    # No solution exists
    return None

def find_solution(content) -> None:

    try:
        instance = decode(content)
        if instance is None:
            print("NO") # malformed input
            return
    except:
        print("NO")
        return

    Rs_old = instance[3]
    instance, gammas_removed = reduce_instance(instance)
    
    try:
        assignment = find_brute_force_solution(instance)
    except:
        print("NO")
        return

    if assignment is None:
        print("NO") # no solution exists
        return

    # Answer YES
    for key in assignment.keys():
        print(f"{key}:{assignment[key]}")
    for key in gammas_removed:
        print(f"{key}:{Rs_old[key][0]}")

In [18]:
# Read input from file
f = open("inputs/test_own.swe", "r")
content = f.read()

find_solution(content)

A:a


In [19]:
# Read input from stdin
content = ""
for line in sys.stdin:
    content += line

find_solution(content)

NO
